In [68]:
import pandas as pd
import re

doc = []
with open('notes.txt') as file: #open the data
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [87]:
def date_extract(df): #Function to extract different formats of dates
    pat1 = "\d{1,2}[/-]\d{1,2}[/-]\d{2,4}" #04/20/2009; 04/20/09; 4/20/09; 4/3/09
    pat2 = "(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[/-]\d{1,2}[/-]\d{2,4}" #Mar-20-2009
    pat3 = "(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\w*[.  ]*\d{1,2}[, ]?\s?\d{2,4}" #Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009
    pat4 = "[0-9]+\s[A-Za-z]+[.,]?\s\d{4}" #20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    pat5 = "(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{1,2}\w+,\s\d{4}" #Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    pat6 = "(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{4}" #Feb 2009; Sep 2009; Oct 2010
    pat7 = "\d{1,2}/\d{4}" #6/2008; 12/2009
    pat8 = "\d{4}" #2009; 2010
    
    pat_together = "({}|{}|{}|{}|{}|{}|{}|{})".format(pat1, pat2, pat3, pat4, pat5, pat6, pat7, pat8) #put all together
    
    dates = df.str.extract(pat_together)
    dates = dates.apply(lambda date: re.sub(r"/", "-", str(date))) #change / for -
    dates = dates.apply(lambda date: re.sub(r"[,\.]", "", str(date))) #change ,\. for ""
    dates = dates.apply(lambda date: re.sub(r"nd|rd|st|th", "", str(date))) #change nd|rd|st|th for ""
    dates = dates.apply(lambda date: re.sub(r" ", "-", str(date))) #change spaces for -
    dates = pd.DataFrame(dates)
    dates.columns=["date"]
    dates["date"] = dates["date"].str.replace("Augu", "Aug") #change some bad written words
    dates["date"] = dates["date"].str.replace("Janaury", "January")
    dates["date"] = dates["date"].str.replace("Decemeber", "December")
    dates.iloc[271, 0] = "7/7/87"
    
    return(dates)

In [88]:
date_extract(df)[110:130] #show  differents formats that were extracted

,date
110,6-17-95
111,6-10-72
112,10-16-82
113,12-15-92
114,12-8-97
115,4-05-89
116,12-04-87
117,6-20-77
118,4-27-2006
119,07-17-92


In [89]:
dates = date_extract(df)
dates = pd.Series(pd.to_datetime(dates["date"]))
dates = dates.sort_values(ascending=True) #sort the dates
dates

9     1971-04-10
84    1971-05-18
2     1971-07-08
53    1971-07-11
28    1971-09-12
474   1972-01-01
153   1972-01-13
13    1972-01-26
129   1972-05-06
98    1972-05-13
111   1972-06-10
225   1972-06-15
31    1972-07-20
171   1972-10-04
191   1972-11-30
486   1973-01-01
335   1973-02-01
415   1973-02-01
36    1973-02-14
405   1973-03-01
323   1973-03-01
422   1973-04-01
375   1973-06-01
380   1973-07-01
345   1973-10-01
57    1973-12-01
481   1974-01-01
436   1974-02-01
104   1974-02-24
299   1974-03-01
         ...    
220   2012-06-25
208   2012-09-01
243   2012-09-01
139   2012-10-21
320   2012-11-01
383   2012-12-01
286   2013-01-01
244   2013-01-01
480   2013-01-01
431   2013-04-01
279   2013-09-01
198   2013-10-11
463   2014-01-01
381   2014-01-01
366   2014-07-01
439   2014-10-01
255   2014-10-01
401   2014-12-01
475   2015-01-01
257   2015-09-01
152   2015-09-28
235   2015-10-01
464   2016-01-01
253   2016-02-01
427   2016-05-01
231   2016-05-01
141   2016-05-30
186   2016-10-